# Day 7: Interview Questions - Foundation Week

## Week 1 - Python for Quantitative Finance

### 🎯 Today's Focus
10 common interview questions covering Week 1 topics:
1. Python/NumPy fundamentals
2. Returns and risk metrics
3. Correlation and covariance
4. Portfolio basics

### Interview Tips
- Talk through your thought process
- Write clean, readable code
- Ask clarifying questions if needed
- Discuss edge cases and limitations

**Author**: ML Quant Finance Mastery | **Difficulty**: Interview Level

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Download real data using yfinance
tickers = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS']
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

print("📥 Downloading data from Yahoo Finance...")
data = yf.download(tickers, start=start_date, end=end_date, progress=False, auto_adjust=True)
prices = data['Close'].dropna()
returns = prices.pct_change().dropna()

print("=" * 60)
print("INTERVIEW QUESTIONS - WEEK 1")
print("=" * 60)
print(f"✅ Data loaded: {prices.shape[0]} days, {len(tickers)} stocks")
print("Ready to practice! Good luck! 🎯\n")

📥 Downloading data from Yahoo Finance...
INTERVIEW QUESTIONS - WEEK 1
✅ Data loaded: 1256 days, 5 stocks
Ready to practice! Good luck! 🎯



---
## Q1: Simple vs Log Returns [5 min]

**Question**: Explain the difference between simple and log returns. When would you use each?

<details>
<summary>💡 Hints</summary>

- Think about multi-period aggregation
- Consider what happens with large returns
- What are the mathematical properties?
</details>

In [2]:
# ==================== ANSWER: SIMPLE vs LOG RETURNS ====================

"""
SIMPLE RETURNS: r_t = (P_t - P_{t-1}) / P_{t-1} = P_t / P_{t-1} - 1

LOG RETURNS: R_t = ln(P_t / P_{t-1}) = ln(P_t) - ln(P_{t-1})

KEY DIFFERENCES:
1. Time Aggregation:
   - Simple: Compound (1+r1)(1+r2)...(1+rn) - 1
   - Log: Sum R1 + R2 + ... + Rn (much easier!)

2. Cross-sectional (Portfolio):
   - Simple: Weighted sum works perfectly
   - Log: Doesn't aggregate linearly across assets

3. Distribution:
   - Log returns are more symmetric & closer to normal
   - Simple returns bounded at -100% (can't lose more than invested)

WHEN TO USE:
- Log returns: Time series analysis, modeling, risk calculations
- Simple returns: Portfolio aggregation, performance reporting
"""

# Demonstration
p = prices['AAPL']
simple_ret = p.pct_change().dropna()
log_ret = np.log(p / p.shift(1)).dropna()

# Multi-period return: 1 year
simple_1y = (1 + simple_ret['2023']).prod() - 1
log_1y = log_ret['2023'].sum()
log_1y_converted = np.exp(log_1y) - 1  # Convert log to simple

print("=" * 60)
print("SIMPLE vs LOG RETURNS DEMONSTRATION")
print("=" * 60)
print(f"\n2023 AAPL Return Calculation:")
print(f"  Simple return (compounded):  {simple_1y*100:.2f}%")
print(f"  Log return (summed):         {log_1y*100:.2f}%")
print(f"  Log converted to simple:     {log_1y_converted*100:.2f}%")
print(f"\n✓ Both methods give same result when properly converted!")

# Show distribution difference
print(f"\nDistribution Statistics:")
print(f"{'Metric':<15} {'Simple':>12} {'Log':>12}")
print("-" * 40)
print(f"{'Mean':<15} {simple_ret.mean()*100:>11.4f}% {log_ret.mean()*100:>11.4f}%")
print(f"{'Std Dev':<15} {simple_ret.std()*100:>11.4f}% {log_ret.std()*100:>11.4f}%")
print(f"{'Skewness':<15} {stats.skew(simple_ret):>12.4f} {stats.skew(log_ret):>12.4f}")
print(f"{'Kurtosis':<15} {stats.kurtosis(simple_ret):>12.4f} {stats.kurtosis(log_ret):>12.4f}")

SIMPLE vs LOG RETURNS DEMONSTRATION

2023 AAPL Return Calculation:
  Simple return (compounded):  49.01%
  Log return (summed):         39.88%
  Log converted to simple:     49.01%

✓ Both methods give same result when properly converted!

Distribution Statistics:
Metric                Simple          Log
----------------------------------------
Mean                 0.0612%      0.0461%
Std Dev              1.7433%      1.7372%
Skewness              0.4684       0.2599
Kurtosis              6.7491       5.7384


---
## Q2: Calculate VaR Without Using Libraries [10 min]

**Question**: Given a series of returns, implement Value at Risk (VaR) calculation using the historical method. Do NOT use any statistical libraries.

In [3]:
# ==================== ANSWER: HISTORICAL VaR ====================

def historical_var(returns, confidence=0.95):
    """
    Calculate Historical VaR without using statistical libraries.
    
    VaR = the loss level that will not be exceeded with (confidence)% probability
    
    For 95% confidence: Find the 5th percentile of returns
    """
    # Sort returns from worst to best
    sorted_returns = sorted(returns)
    
    # Find the index for the (1-confidence) percentile
    n = len(sorted_returns)
    index = int((1 - confidence) * n)
    
    # VaR is the return at that index (negative return = loss)
    var = -sorted_returns[index]  # Convert to positive loss
    
    return var

def historical_es(returns, confidence=0.95):
    """
    Calculate Expected Shortfall (Conditional VaR)
    ES = Average of returns worse than VaR
    """
    sorted_returns = sorted(returns)
    n = len(sorted_returns)
    cutoff_index = int((1 - confidence) * n)
    
    # Average of all returns in the tail
    tail_returns = sorted_returns[:cutoff_index + 1]
    es = -np.mean(tail_returns)
    
    return es

# Test with AAPL returns
aapl_ret = returns['AAPL'].values

var_95 = historical_var(aapl_ret, 0.95)
var_99 = historical_var(aapl_ret, 0.99)
es_95 = historical_es(aapl_ret, 0.95)
es_99 = historical_es(aapl_ret, 0.99)

print("=" * 60)
print("HISTORICAL VaR CALCULATION (No Libraries)")
print("=" * 60)
print(f"\nAAPL Daily Returns ({len(aapl_ret)} observations)")
print(f"\n{'Metric':<20} {'95% Conf':>12} {'99% Conf':>12}")
print("-" * 46)
print(f"{'VaR':<20} {var_95*100:>11.2f}% {var_99*100:>11.2f}%")
print(f"{'Expected Shortfall':<20} {es_95*100:>11.2f}% {es_99*100:>11.2f}%")

print(f"\n💡 Interpretation:")
print(f"  - With 95% confidence, daily loss won't exceed {var_95*100:.2f}%")
print(f"  - If loss exceeds VaR, average loss is {es_95*100:.2f}% (ES)")

# Verify against numpy percentile
print(f"\n✓ Verification: np.percentile gives {-np.percentile(aapl_ret, 5)*100:.2f}%")

HISTORICAL VaR CALCULATION (No Libraries)

AAPL Daily Returns (1255 observations)

Metric                   95% Conf     99% Conf
----------------------------------------------
VaR                         2.84%        4.24%
Expected Shortfall          3.89%        5.55%

💡 Interpretation:
  - With 95% confidence, daily loss won't exceed 2.84%
  - If loss exceeds VaR, average loss is 3.89% (ES)

✓ Verification: np.percentile gives 2.80%


---
## Q3: Why Does Diversification Reduce Risk? [5 min]

**Question**: Mathematically explain why adding uncorrelated assets reduces portfolio volatility.

In [4]:
# ==================== ANSWER: DIVERSIFICATION MATH ====================

"""
PORTFOLIO VARIANCE FORMULA:
σ²_p = Σᵢ Σⱼ wᵢwⱼσᵢσⱼρᵢⱼ

For 2 assets with equal weights (w=0.5):
σ²_p = 0.25σ₁² + 0.25σ₂² + 0.5ρ₁₂σ₁σ₂

KEY INSIGHT:
- If ρ = 1: No diversification benefit (perfect correlation)
- If ρ = 0: σ²_p = 0.25(σ₁² + σ₂²) → Portfolio vol < average of individual vols
- If ρ = -1: Can potentially eliminate all risk!

The cross-term (2w₁w₂ρ₁₂σ₁σ₂) is what drives diversification benefit.
Lower correlation → Lower cross-term → Lower portfolio variance.
"""

# Numerical demonstration
vol1, vol2 = 0.20, 0.25  # 20% and 25% annual volatility
w1, w2 = 0.5, 0.5

print("=" * 60)
print("DIVERSIFICATION: MATHEMATICAL PROOF")
print("=" * 60)
print(f"\nAsset 1 Volatility: {vol1*100:.0f}%")
print(f"Asset 2 Volatility: {vol2*100:.0f}%")
print(f"Average Volatility: {(vol1+vol2)/2*100:.1f}%")
print(f"\nEqual-Weighted Portfolio Volatility by Correlation:")
print("-" * 50)
print(f"{'Correlation':>12} {'Port Vol':>12} {'Diversification':>18}")
print("-" * 50)

for rho in [1.0, 0.8, 0.5, 0.2, 0.0, -0.5, -1.0]:
    # Portfolio variance = w1²σ1² + w2²σ2² + 2w1w2ρσ1σ2
    port_var = w1**2 * vol1**2 + w2**2 * vol2**2 + 2*w1*w2*rho*vol1*vol2
    port_vol = np.sqrt(port_var)
    avg_vol = (vol1 + vol2) / 2
    div_benefit = (avg_vol - port_vol) / avg_vol * 100
    
    print(f"{rho:>12.1f} {port_vol*100:>11.1f}% {div_benefit:>17.1f}%")

print("\n💡 Key Insight: With ρ = -1, portfolio volatility can be ZERO!")
print("   This is why correlation is crucial for portfolio construction.")

DIVERSIFICATION: MATHEMATICAL PROOF

Asset 1 Volatility: 20%
Asset 2 Volatility: 25%
Average Volatility: 22.5%

Equal-Weighted Portfolio Volatility by Correlation:
--------------------------------------------------
 Correlation     Port Vol    Diversification
--------------------------------------------------
         1.0        22.5%               0.0%
         0.8        21.4%               5.1%
         0.5        19.5%              13.2%
         0.2        17.5%              22.2%
         0.0        16.0%              28.9%
        -0.5        11.5%              49.1%
        -1.0         2.5%              88.9%

💡 Key Insight: With ρ = -1, portfolio volatility can be ZERO!
   This is why correlation is crucial for portfolio construction.


---
## Q4: Implement Sharpe Ratio [5 min]

**Question**: Write a function to calculate the Sharpe Ratio. What are its limitations?

In [5]:
# ==================== ANSWER: SHARPE RATIO ====================

def sharpe_ratio(returns, rf_rate=0.05, periods_per_year=252):
    """
    Calculate annualized Sharpe Ratio
    
    Sharpe = (R_p - R_f) / σ_p
    
    Where:
    - R_p = Portfolio return (annualized)
    - R_f = Risk-free rate (annualized)
    - σ_p = Portfolio volatility (annualized)
    """
    # Annualize returns
    mean_return = np.mean(returns) * periods_per_year
    
    # Annualize volatility
    volatility = np.std(returns, ddof=1) * np.sqrt(periods_per_year)
    
    # Sharpe ratio
    sharpe = (mean_return - rf_rate) / volatility
    
    return sharpe

# Calculate for our stocks
print("=" * 60)
print("SHARPE RATIO CALCULATION")
print("=" * 60)
print(f"\n{'Asset':<10} {'Ann Return':>12} {'Ann Vol':>12} {'Sharpe':>10}")
print("-" * 46)

for ticker in ['AAPL', 'MSFT', 'GOOGL']:
    ret = returns[ticker]
    ann_ret = ret.mean() * 252
    ann_vol = ret.std() * np.sqrt(252)
    sharpe = sharpe_ratio(ret.values)
    print(f"{ticker:<10} {ann_ret*100:>11.1f}% {ann_vol*100:>11.1f}% {sharpe:>10.2f}")

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
LIMITATIONS OF SHARPE RATIO:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. ASSUMES NORMAL RETURNS
   - Doesn't account for skewness or fat tails
   - Strategies with negative skew look better than they are

2. PENALIZES UPSIDE VOLATILITY
   - Treats gains and losses symmetrically
   - Sortino ratio (uses downside deviation) addresses this

3. SENSITIVE TO TIME PERIOD
   - Different periods give different Sharpes
   - Annualization assumes i.i.d. returns

4. IGNORES LIQUIDITY & CAPACITY
   - High Sharpe strategies may not scale
   - Transaction costs not included

5. MANIPULATION
   - Can be gamed by selling OTM options
   - Illiquid assets may show artificially high Sharpe
""")

SHARPE RATIO CALCULATION

Asset        Ann Return      Ann Vol     Sharpe
----------------------------------------------
AAPL              15.4%        27.7%       0.38
MSFT              18.3%        25.7%       0.52
GOOGL             29.9%        31.0%       0.80

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
LIMITATIONS OF SHARPE RATIO:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. ASSUMES NORMAL RETURNS
   - Doesn't account for skewness or fat tails
   - Strategies with negative skew look better than they are

2. PENALIZES UPSIDE VOLATILITY
   - Treats gains and losses symmetrically
   - Sortino ratio (uses downside deviation) addresses this

3. SENSITIVE TO TIME PERIOD
   - Different periods give different Sharpes
   - Annualization assumes i.i.d. returns

4. IGNORES LIQUIDITY & CAPACITY
   - High Sharpe strategies may not scale
   - Transaction costs not included

5. MANIPULATION
   - Can be gamed by selling OTM options
   - Illiquid assets may sh

---
## Q5: Look-Ahead Bias [5 min]

**Question**: What is look-ahead bias? Give an example and explain how to avoid it.

In [6]:
# ==================== ANSWER: LOOK-AHEAD BIAS ====================

"""
LOOK-AHEAD BIAS: Using information that would not have been available
at the time a trading decision was made.

COMMON EXAMPLES:
1. Using earnings data before it was announced
2. Calculating rolling averages using future data points
3. Optimizing strategy parameters on full historical data

This leads to OVERFITTING and strategies that work in backtests
but fail in live trading.
"""

print("=" * 60)
print("LOOK-AHEAD BIAS DEMONSTRATION")
print("=" * 60)

# Example: Trading signal using adjusted prices
# WRONG WAY (look-ahead bias)
def wrong_momentum_signal(prices):
    """Uses future-adjusted prices that weren't known at the time"""
    # This centered rolling mean uses FUTURE data points!
    return prices.rolling(20, center=True).mean()

# CORRECT WAY (point-in-time)
def correct_momentum_signal(prices):
    """Uses only past data available at decision time"""
    # Trailing rolling mean - only uses past 20 days
    return prices.rolling(20, center=False).mean()

# Demonstration
demo_prices = prices['AAPL']['2023-01-01':'2023-03-01']

wrong_signal = wrong_momentum_signal(demo_prices)
correct_signal = correct_momentum_signal(demo_prices)

print(f"\nOn 2023-02-01, calculating 20-day moving average:")
print(f"  ❌ WRONG (centered):  Uses data from Jan 22 to Feb 10 (FUTURE!)")
print(f"  ✓ CORRECT (trailing): Uses data from Jan 12 to Feb 1 only")

print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
HOW TO AVOID LOOK-AHEAD BIAS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. USE POINT-IN-TIME DATA
   - Store data snapshots as they were available
   - Account for announcement delays (earnings, economic data)

2. SHIFT SIGNALS APPROPRIATELY
   - If signal calculated at close, can only trade next day
   - Add realistic execution delays

3. AVOID CENTERED ROLLING WINDOWS
   - Always use trailing windows: rolling(window, center=False)

4. STRICT TRAIN/TEST SPLIT
   - Never train on test period data
   - Use walk-forward optimization

5. PAPER TRADE BEFORE LIVE
   - Real-time paper trading catches many data issues

Example Code Pattern:
  # Good: Shift signal by 1 day for realistic execution
  signal = data.rolling(20).mean().shift(1)
  position = np.where(data > signal, 1, 0)  # Position known at close
  returns = position.shift(1) * data.pct_change()  # Trade next day
""")

LOOK-AHEAD BIAS DEMONSTRATION

On 2023-02-01, calculating 20-day moving average:
  ❌ WRONG (centered):  Uses data from Jan 22 to Feb 10 (FUTURE!)
  ✓ CORRECT (trailing): Uses data from Jan 12 to Feb 1 only

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
HOW TO AVOID LOOK-AHEAD BIAS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. USE POINT-IN-TIME DATA
   - Store data snapshots as they were available
   - Account for announcement delays (earnings, economic data)

2. SHIFT SIGNALS APPROPRIATELY
   - If signal calculated at close, can only trade next day
   - Add realistic execution delays

3. AVOID CENTERED ROLLING WINDOWS
   - Always use trailing windows: rolling(window, center=False)

4. STRICT TRAIN/TEST SPLIT
   - Never train on test period data
   - Use walk-forward optimization

5. PAPER TRADE BEFORE LIVE
   - Real-time paper trading catches many data issues

Example Code Pattern:
  # Good: Shift signal by 1 day for realistic execution
  signal = dat

---
## Q6: Calculate Correlation from Scratch [10 min]

**Question**: Implement Pearson correlation coefficient without using numpy.corrcoef or pandas.corr().

In [7]:
# ==================== ANSWER: CORRELATION FROM SCRATCH ====================

def correlation(x, y):
    """
    Calculate Pearson correlation coefficient from scratch.
    
    Formula: ρ = Cov(X,Y) / (σ_X * σ_Y)
           = Σ(x-μx)(y-μy) / sqrt(Σ(x-μx)² * Σ(y-μy)²)
    """
    n = len(x)
    
    # Step 1: Calculate means
    mean_x = sum(x) / n
    mean_y = sum(y) / n
    
    # Step 2: Calculate deviations from mean
    dev_x = [xi - mean_x for xi in x]
    dev_y = [yi - mean_y for yi in y]
    
    # Step 3: Calculate covariance numerator
    cov_num = sum(dx * dy for dx, dy in zip(dev_x, dev_y))
    
    # Step 4: Calculate standard deviations
    var_x = sum(dx ** 2 for dx in dev_x)
    var_y = sum(dy ** 2 for dy in dev_y)
    
    # Step 5: Calculate correlation
    denominator = (var_x * var_y) ** 0.5
    
    if denominator == 0:
        return 0  # Handle edge case
    
    corr = cov_num / denominator
    
    return corr

# Test our implementation
x = returns['AAPL'].values
y = returns['MSFT'].values

our_corr = correlation(x, y)
numpy_corr = np.corrcoef(x, y)[0, 1]
pandas_corr = returns['AAPL'].corr(returns['MSFT'])

print("=" * 60)
print("CORRELATION FROM SCRATCH")
print("=" * 60)
print(f"\nAAPL vs MSFT Correlation:")
print(f"  Our implementation: {our_corr:.6f}")
print(f"  NumPy corrcoef:     {numpy_corr:.6f}")
print(f"  Pandas corr:        {pandas_corr:.6f}")
print(f"\n✓ All methods match!")

# Also show the formula breakdown
print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
FORMULA BREAKDOWN:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Pearson Correlation:
                   Σ(x - μx)(y - μy)
    ρ = ─────────────────────────────────────
         √[Σ(x - μx)² × Σ(y - μy)²]

Which is equivalent to:
           Cov(X, Y)
    ρ = ─────────────
          σx × σy

Properties:
• -1 ≤ ρ ≤ 1
• ρ = 1: Perfect positive linear relationship
• ρ = -1: Perfect negative linear relationship
• ρ = 0: No linear relationship (NOT independence!)
""")

CORRELATION FROM SCRATCH

AAPL vs MSFT Correlation:
  Our implementation: 0.626692
  NumPy corrcoef:     0.626692
  Pandas corr:        0.626692

✓ All methods match!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
FORMULA BREAKDOWN:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Pearson Correlation:
                   Σ(x - μx)(y - μy)
    ρ = ─────────────────────────────────────
         √[Σ(x - μx)² × Σ(y - μy)²]

Which is equivalent to:
           Cov(X, Y)
    ρ = ─────────────
          σx × σy

Properties:
• -1 ≤ ρ ≤ 1
• ρ = 1: Perfect positive linear relationship
• ρ = -1: Perfect negative linear relationship
• ρ = 0: No linear relationship (NOT independence!)



---
## Q7: Maximum Drawdown [5 min]

**Question**: Calculate maximum drawdown for a price series. What does it represent?

In [8]:
# ==================== ANSWER: MAXIMUM DRAWDOWN ====================

def max_drawdown(prices):
    """
    Calculate maximum drawdown - the worst peak-to-trough decline.
    
    Drawdown at time t = (P_t - Peak_t) / Peak_t
    Max Drawdown = min(Drawdown_t) for all t
    """
    # Running maximum (peak)
    peak = prices.expanding(min_periods=1).max()
    
    # Drawdown series
    drawdown = (prices - peak) / peak
    
    # Maximum drawdown
    max_dd = drawdown.min()
    
    # Find dates
    max_dd_date = drawdown.idxmin()
    peak_date = prices.loc[:max_dd_date].idxmax()
    
    return max_dd, peak_date, max_dd_date

# Calculate for our assets
print("=" * 60)
print("MAXIMUM DRAWDOWN ANALYSIS")
print("=" * 60)
print(f"\n{'Asset':<10} {'Max DD':>12} {'Peak Date':>15} {'Trough Date':>15}")
print("-" * 55)

for ticker in ['AAPL', 'MSFT', 'GOOGL']:
    max_dd, peak_date, trough_date = max_drawdown(prices[ticker])
    print(f"{ticker:<10} {max_dd*100:>11.1f}% {str(peak_date.date()):>15} {str(trough_date.date()):>15}")

print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
WHY MAX DRAWDOWN MATTERS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. INVESTOR PSYCHOLOGY
   - Shows worst-case scenario for investors
   - Many quit during drawdowns (sell at worst time)

2. RISK ASSESSMENT  
   - Complements volatility (which is symmetric)
   - Shows actual downside realized

3. STRATEGY EVALUATION
   - High Sharpe but deep drawdown = dangerous
   - Calmar Ratio = Ann Return / Max Drawdown

4. CAPACITY CONSTRAINTS
   - Deep drawdowns may trigger redemptions
   - May force deleveraging at worst time
""")

MAXIMUM DRAWDOWN ANALYSIS

Asset            Max DD       Peak Date     Trough Date
-------------------------------------------------------
AAPL             -33.4%      2024-12-26      2025-04-08
MSFT             -37.1%      2021-11-19      2022-11-03
GOOGL            -44.3%      2021-11-18      2022-11-03

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
WHY MAX DRAWDOWN MATTERS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. INVESTOR PSYCHOLOGY
   - Shows worst-case scenario for investors
   - Many quit during drawdowns (sell at worst time)

2. RISK ASSESSMENT  
   - Complements volatility (which is symmetric)
   - Shows actual downside realized

3. STRATEGY EVALUATION
   - High Sharpe but deep drawdown = dangerous
   - Calmar Ratio = Ann Return / Max Drawdown

4. CAPACITY CONSTRAINTS
   - Deep drawdowns may trigger redemptions
   - May force deleveraging at worst time



---
## Q8: Quick-Fire Questions [Rapid]

**Answer these verbally in 30 seconds each:**

In [9]:
# ==================== QUICK-FIRE Q&A ====================

quick_fire = """
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUICK-FIRE INTERVIEW QUESTIONS & ANSWERS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Q: What's the difference between correlation and covariance?
A: Correlation is normalized (-1 to 1), covariance depends on scale.
   Correlation = Cov(X,Y) / (σX × σY)

Q: What is survivorship bias?
A: Only analyzing assets that still exist, excluding failures.
   Leads to overestimated historical returns.

Q: How many trading days in a year?
A: 252 (for US markets)

Q: What does a beta of 1.5 mean?
A: Asset moves 1.5x the market. If market +1%, asset expected +1.5%

Q: Difference between systematic and idiosyncratic risk?
A: Systematic: Market-wide, cannot diversify away (beta exposure)
   Idiosyncratic: Company-specific, CAN diversify away

Q: What is alpha?
A: Excess return beyond what beta exposure would explain.
   R = α + β×Rm + ε

Q: Why sqrt(252) to annualize daily volatility?
A: Variance scales linearly with time (if i.i.d.).
   Vol = √Variance, so Vol scales with √time.
   Daily Vol × √252 = Annual Vol

Q: What's a good Sharpe ratio?
A: < 0.5: Poor
   0.5-1.0: Acceptable
   1.0-2.0: Good
   > 2.0: Excellent (may be too good - check for data issues!)

Q: Why do correlations increase during crises?
A: Flight to safety, forced liquidations, contagion effects.
   Diversification fails when most needed.

Q: What is the information ratio?
A: Sharpe ratio of active returns (vs benchmark).
   IR = (R_portfolio - R_benchmark) / Tracking Error
"""
print(quick_fire)


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUICK-FIRE INTERVIEW QUESTIONS & ANSWERS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Q: What's the difference between correlation and covariance?
A: Correlation is normalized (-1 to 1), covariance depends on scale.
   Correlation = Cov(X,Y) / (σX × σY)

Q: What is survivorship bias?
A: Only analyzing assets that still exist, excluding failures.
   Leads to overestimated historical returns.

Q: How many trading days in a year?
A: 252 (for US markets)

Q: What does a beta of 1.5 mean?
A: Asset moves 1.5x the market. If market +1%, asset expected +1.5%

Q: Difference between systematic and idiosyncratic risk?
A: Systematic: Market-wide, cannot diversify away (beta exposure)
   Idiosyncratic: Company-specific, CAN diversify away

Q: What is alpha?
A: Excess return beyond what beta exposure would explain.
   R = α + β×Rm + ε

Q: Why sqrt(252) to annualize daily volatility?
A: Variance scales linearly with time (

---
## 🎓 Week 1 Complete!

### What We Covered This Week
1. **Python/NumPy**: Vectorized operations, financial arrays
2. **Pandas**: Time series, point-in-time data, resampling
3. **Returns**: Simple vs log, daily to annual
4. **Risk Metrics**: VaR, ES, volatility, drawdowns
5. **Correlation**: Matrices, rolling correlations, PCA
6. **Visualization**: Professional financial charts
7. **Portfolio Basics**: Variance, diversification, optimization

### Interview Readiness Checklist ✅
- [ ] Can calculate returns and volatility from scratch
- [ ] Understand VaR/ES and their limitations
- [ ] Can explain diversification mathematically
- [ ] Know Sharpe ratio formula and limitations
- [ ] Can identify look-ahead bias
- [ ] Understand correlation vs covariance
- [ ] Can discuss maximum drawdown

### Next Week Preview: Statistics & Probability
- Probability distributions (Normal, Student-t, etc.)
- Hypothesis testing for trading strategies
- Statistical arbitrage foundations
- Regression and factor models

**Congratulations on completing Week 1! 🎉**

## 🚀 TODAY'S TRADING SIGNAL

Applying interview concepts to generate real-world trading insights.

In [10]:
# =============================================================================
# 🚀 TODAY'S TRADING SIGNAL - Interview Fundamentals Analysis
# =============================================================================

print("=" * 70)
print("📊 TODAY'S TRADING ANALYSIS - Using Interview Question Concepts")
print("=" * 70)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print()

# Current prices
latest_prices = prices.iloc[-1]
print("📈 CURRENT MARKET SNAPSHOT:")
print("-" * 50)
for ticker in tickers:
    print(f"   {ticker}: ${latest_prices[ticker]:.2f}")

# Calculate metrics from interview questions
print("\n📊 KEY METRICS (Interview Question Applications):")
print("-" * 50)

# 20-day momentum (from returns question)
momentum_20d = (prices.iloc[-1] / prices.iloc[-20] - 1) * 100
print("\n🔹 20-Day Momentum (Simple Returns):")
for ticker in tickers:
    direction = "🟢" if momentum_20d[ticker] > 0 else "🔴"
    print(f"   {direction} {ticker}: {momentum_20d[ticker]:+.2f}%")

# Volatility (60-day annualized)
vol_60d = returns.iloc[-60:].std() * np.sqrt(252) * 100
print("\n🔹 60-Day Volatility (Annualized):")
for ticker in tickers:
    vol_level = "🟢 LOW" if vol_60d[ticker] < 20 else ("🟡 MEDIUM" if vol_60d[ticker] < 30 else "🔴 HIGH")
    print(f"   {ticker}: {vol_60d[ticker]:.1f}% ({vol_level})")

# VaR and ES (from risk question)
print("\n🔹 Value at Risk (95%, 1-day):")
for ticker in tickers:
    var_95 = np.percentile(returns[ticker].dropna(), 5) * 100
    print(f"   {ticker}: {var_95:.2f}% (potential daily loss)")

# Sharpe Ratio
rf_rate = 0.05
ret_60d = returns.iloc[-60:].mean() * 252
sharpe_60d = (ret_60d - rf_rate) / (returns.iloc[-60:].std() * np.sqrt(252))
print("\n🔹 60-Day Sharpe Ratio:")
for ticker in tickers:
    sharpe_level = "🟢 STRONG" if sharpe_60d[ticker] > 1 else ("🟡 MODERATE" if sharpe_60d[ticker] > 0 else "🔴 WEAK")
    print(f"   {ticker}: {sharpe_60d[ticker]:.2f} ({sharpe_level})")

# Generate trading signals
print("\n" + "=" * 70)
print("🎯 TRADING SIGNALS FOR TODAY")
print("=" * 70)

for ticker in tickers:
    mom = momentum_20d[ticker]
    vol = vol_60d[ticker]
    sharpe = sharpe_60d[ticker]
    
    # Score-based signal
    score = 0
    if mom > 5: score += 2
    elif mom > 0: score += 1
    elif mom < -5: score -= 2
    else: score -= 1
    
    if sharpe > 1: score += 2
    elif sharpe > 0: score += 1
    else: score -= 1
    
    if vol < 20: score += 1
    elif vol > 35: score -= 1
    
    # Determine signal
    if score >= 3:
        signal = "🟢 STRONG BUY"
        action = "Consider buying shares or CALL options"
    elif score >= 1:
        signal = "🟢 WEAK BUY"
        action = "Consider small long position or CALL spreads"
    elif score <= -3:
        signal = "🔴 STRONG SELL"
        action = "Consider selling or protective PUTs"
    elif score <= -1:
        signal = "🟠 WEAK SELL"
        action = "Consider trimming position or PUT spreads"
    else:
        signal = "⚪ HOLD/NEUTRAL"
        action = "No clear edge - stay on sidelines or hold existing"
    
    print(f"\n{'='*30} {ticker} {'='*30}")
    print(f"   Signal: {signal}")
    print(f"   Action: {action}")
    print(f"   Reasoning: Momentum={mom:+.1f}%, Vol={vol:.1f}%, Sharpe={sharpe:.2f}")

print("\n" + "=" * 70)
print("⚠️ DISCLAIMER: This is educational analysis, not financial advice.")
print("   Always do your own research and consider your risk tolerance.")
print("=" * 70)

📊 TODAY'S TRADING ANALYSIS - Using Interview Question Concepts
Analysis Date: 2026-01-24 09:47

📈 CURRENT MARKET SNAPSHOT:
--------------------------------------------------
   AAPL: $248.04
   MSFT: $465.95
   GOOGL: $327.93
   JPM: $297.72
   GS: $918.88

📊 KEY METRICS (Interview Question Applications):
--------------------------------------------------

🔹 20-Day Momentum (Simple Returns):
   🔴 AAPL: -9.41%
   🔴 MSFT: -4.52%
   🟢 GOOGL: +4.41%
   🔴 JPM: -9.15%
   🟢 GS: +0.89%

🔹 60-Day Volatility (Annualized):
   AAPL: 14.5% (🟢 LOW)
   MSFT: 21.5% (🟡 MEDIUM)
   GOOGL: 28.5% (🟡 MEDIUM)
   JPM: 24.5% (🟡 MEDIUM)
   GS: 27.6% (🟡 MEDIUM)

🔹 Value at Risk (95%, 1-day):
   AAPL: -2.80% (potential daily loss)
   MSFT: -2.64% (potential daily loss)
   GOOGL: -2.94% (potential daily loss)
   JPM: -2.33% (potential daily loss)
   GS: -2.40% (potential daily loss)

🔹 60-Day Sharpe Ratio:
   AAPL: -2.58 (🔴 WEAK)
   MSFT: -2.66 (🔴 WEAK)
   GOOGL: 2.88 (🟢 STRONG)
   JPM: -0.37 (🔴 WEAK)
   GS: 2.34 

## 🎯 Key Takeaways for Interview Success

### 1. Master the Fundamentals
| Topic | Must-Know Concepts |
|-------|-------------------|
| **Returns** | Log vs simple returns, compounding, annualization |
| **Risk Metrics** | Volatility, VaR, CVaR, drawdown, Sharpe/Sortino |
| **Correlation** | Pearson vs Spearman, correlation breakdown in crises |
| **Portfolio Theory** | Mean-variance, efficient frontier, diversification |

### 2. Common Interview Pitfalls to Avoid
- ❌ Confusing **correlation with causation**
- ❌ Using simple returns for long horizons (use log returns)
- ❌ Ignoring **non-stationarity** in financial data
- ❌ Assuming normal distributions (fat tails matter!)
- ❌ Overlooking **survivorship bias** in backtests
- ❌ Forgetting **transaction costs** in strategy analysis

### 3. Framework for Answering Technical Questions
```
1. CLARIFY: Restate the problem, ask about constraints
2. STRUCTURE: Outline your approach before diving in
3. SOLVE: Work through methodically, state assumptions
4. VERIFY: Sanity check results, discuss edge cases
5. EXTEND: Mention limitations and improvements
```

### 4. Key Formulas to Memorize
| Formula | When to Use |
|---------|-------------|
| $r_{log} = \ln(P_t / P_{t-1})$ | Multi-period returns, log-normal models |
| $\sigma_{annual} = \sigma_{daily} \times \sqrt{252}$ | Annualizing volatility |
| $Sharpe = \frac{r_p - r_f}{\sigma_p}$ | Risk-adjusted performance |
| $VaR_\alpha = \mu - z_\alpha \sigma$ | Parametric Value at Risk |
| $\rho_{X,Y} = \frac{Cov(X,Y)}{\sigma_X \sigma_Y}$ | Correlation coefficient |

### 5. Behavioral Tips
- ✅ **Think out loud** - interviewers want to see your thought process
- ✅ **Ask clarifying questions** - shows attention to detail
- ✅ **Acknowledge uncertainty** - "I would verify this by..."
- ✅ **Connect to real applications** - "In practice, I would also consider..."
- ✅ **Be honest about gaps** - "I'm not sure, but my intuition is..."

### 6. Week 1 Concepts You MUST Be Able to Explain
1. Why do we use **log returns** instead of simple returns?
2. What's the difference between **volatility** and **variance**?
3. How does **correlation** affect portfolio risk?
4. What is the **efficient frontier** and why does it matter?
5. Explain **VaR** and its limitations
6. What causes **correlation breakdown** during market stress?
7. How would you detect **outliers** in financial data?

### 7. Practice Makes Perfect
- 📝 Write out solutions by hand before coding
- 🗣️ Practice explaining concepts verbally (record yourself)
- ⏱️ Time yourself on practice problems (aim for efficiency)
- 🔄 Review mistakes and create flashcards for weak areas

---

**Remember:** Interviewers value **clear thinking** over perfect answers. Structure your response, communicate your reasoning, and show intellectual curiosity!